<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
#Imports

import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras.layers import Dropout

Using TensorFlow backend.


In [33]:
# load dataset
dataframe = pandas.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv", delimiter=',', na_values=[" "])
dataframe = dataframe.dropna()

In [34]:
print ("Rows     : " ,dataframe.shape[0])
print ("Columns  : " ,dataframe.shape[1])
print ("\nFeatures : \n" ,dataframe.columns.tolist())
print ("\nMissing values :  ", dataframe.isnull().sum().values.sum())
print ("\nUnique values :  \n",dataframe.nunique())

Rows     :  7032
Columns  :  21

Features : 
 ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

Missing values :   0

Unique values :  
 customerID          7032
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                72
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1584
TotalCharges        6530
Churn                  2
dtype: int64


In [39]:
import numpy as np

#Replacing spaces with null values in total charges column
dataframe['TotalCharges'] = dataframe["TotalCharges"].replace(" ",np.nan)

#Dropping null values from total charges column which contain .15% missing data 
dataframe = dataframe[dataframe["TotalCharges"].notnull()]
dataframe = dataframe.reset_index()[dataframe.columns]

#convert to float type
dataframe["TotalCharges"] = dataframe["TotalCharges"].astype(float)

#replace 'No internet service' to No for the following columns
replace_cols = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']
for i in replace_cols : 
    dataframe[i]  = dataframe[i].replace({'No internet service' : 'No'})
    
#replace values
dataframe["SeniorCitizen"] = dataframe["SeniorCitizen"].replace({1:"Yes",0:"No"})

#Tenure to categorical column
def tenure_lab(dataframe) :
    
    if dataframe["tenure"] <= 12 :
        return "Tenure_0-12"
    elif (dataframe["tenure"] > 12) & (dataframe["tenure"] <= 24 ):
        return "Tenure_12-24"
    elif (dataframe["tenure"] > 24) & (dataframe["tenure"] <= 48) :
        return "Tenure_24-48"
    elif (dataframe["tenure"] > 48) & (dataframe["tenure"] <= 60) :
        return "Tenure_48-60"
    elif dataframe["tenure"] > 60 :
        return "Tenure_gt_60"
dataframe["tenure_group"] = dataframe.apply(lambda dataframe:tenure_lab(dataframe),
                                      axis = 1)

#Separating churn and non churn customers
churn     = dataframe[dataframe["Churn"] == "Yes"]
not_churn = dataframe[dataframe["Churn"] == "No"]

#Separating catagorical and numerical columns
Id_col     = ['customerID']
target_col = ["Churn"]
cat_cols   = dataframe.nunique()[dataframe.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
num_cols   = [x for x in dataframe.columns if x not in cat_cols + target_col + Id_col]

In [44]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd

#customer id col
Id_col     = ['customerID']
#Target columns
target_col = ["Churn"]
#categorical columns
cat_cols   = dataframe.nunique()[dataframe.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
#numerical columns
num_cols   = [x for x in dataframe.columns if x not in cat_cols + target_col + Id_col]
#Binary columns with 2 values
bin_cols   = dataframe.nunique()[dataframe.nunique() == 2].keys().tolist()
#Columns more than 2 values
multi_cols = [i for i in cat_cols if i not in bin_cols]

#Label encoding Binary columns
le = LabelEncoder()
for i in bin_cols :
    dataframe[i] = le.fit_transform(dataframe[i])
    
#Duplicating columns for multi value columns
dataframe = pd.get_dummies(data = dataframe,columns = multi_cols )

#Scaling Numerical columns
std = StandardScaler()
scaled = std.fit_transform(dataframe[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

#dropping original values merging scaled values for numerical columns
df_telcom_og = dataframe.copy()
dataframe = dataframe.drop(columns = num_cols,axis = 1)
dataframe = dataframe.merge(scaled,left_index=True,right_index=True,how = "left")


In [48]:
dataframe.drop("customerID", axis= 1, inplace= True)
dataset = dataframe.values
dataframe.dtypes

gender                                       int64
SeniorCitizen                                int64
Partner                                      int64
Dependents                                   int64
PhoneService                                 int64
OnlineSecurity                               int64
OnlineBackup                                 int64
DeviceProtection                             int64
TechSupport                                  int64
StreamingTV                                  int64
StreamingMovies                              int64
PaperlessBilling                             int64
Churn                                        int64
MultipleLines_No                             uint8
MultipleLines_No phone service               uint8
MultipleLines_Yes                            uint8
InternetService_DSL                          uint8
InternetService_Fiber optic                  uint8
InternetService_No                           uint8
Contract_Month-to-month        

In [49]:
# split into input (X) and output (Y) variables

from sklearn.model_selection import train_test_split

X = dataset[:,0:20]
y = dataset[:,20]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [52]:
# Important Hyperparameters
inputs = X_train.shape[1]
epochs = 5
batch_size = 10


# Create Model
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(inputs,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile Model
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'binary_accuracy'])

# Fit Model
model.fit(X_train, y_train, 
          validation_data=(X_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )

Train on 4711 samples, validate on 2321 samples
Epoch 1/5
4711/4711 [==============================] - 1s 192us/step - loss: 0.1357 - mse: 0.1357 - binary_accuracy: 0.7939 - val_loss: 0.1014 - val_mse: 0.1014 - val_binary_accuracy: 0.8341
Epoch 2/5
4711/4711 [==============================] - 1s 144us/step - loss: 0.0982 - mse: 0.0982 - binary_accuracy: 0.8431 - val_loss: 0.0912 - val_mse: 0.0912 - val_binary_accuracy: 0.8630
Epoch 3/5
4711/4711 [==============================] - 1s 143us/step - loss: 0.0943 - mse: 0.0943 - binary_accuracy: 0.8459 - val_loss: 0.0959 - val_mse: 0.0959 - val_binary_accuracy: 0.8514
Epoch 4/5
4711/4711 [==============================] - 1s 166us/step - loss: 0.0934 - mse: 0.0934 - binary_accuracy: 0.8527 - val_loss: 0.0906 - val_mse: 0.0906 - val_binary_accuracy: 0.8587
Epoch 5/5
4711/4711 [==============================] - 1s 157us/step - loss: 0.0918 - mse: 0.0918 - binary_accuracy: 0.8544 - val_loss: 0.0915 - val_mse: 0.0915 - val_binary_accuracy: 0.85

In [57]:
# Function to create model, required for KerasClassifier
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model():
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(inputs,)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='mse', metrics=['mse', 'accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X, y)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.8526734908421835 using {'batch_size': 40, 'epochs': 20}
Means: 0.8408703009287516, Stdev: 0.009753379008365527 with: {'batch_size': 10, 'epochs': 20}
Means: 0.8478384613990784, Stdev: 0.006538429044190065 with: {'batch_size': 20, 'epochs': 20}
Means: 0.8526734908421835, Stdev: 0.007869070447583835 with: {'batch_size': 40, 'epochs': 20}
Means: 0.8494027455647787, Stdev: 0.002118840829979028 with: {'batch_size': 60, 'epochs': 20}
Means: 0.8516780336697897, Stdev: 0.0068288958474473395 with: {'batch_size': 80, 'epochs': 20}
Means: 0.8515358368555704, Stdev: 0.008750056300480384 with: {'batch_size': 100, 'epochs': 20}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?